# Chess Engine Training on Google Colab

This notebook will help you train the chess engine using Google Colab's GPU. Follow these steps:

1. Mount your Google Drive to save model checkpoints
2. Install required packages
3. Set up the project structure
4. Train the model
5. Download the trained model

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install required packages
!pip install python-chess torch numpy tqdm matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 43.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.7 MB/s eta 0:00:00
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147776 sha25

In [21]:
# Create project structure
!mkdir -p src models

# Create __init__.py
!touch src/__init__.py

In [22]:
# Create board.py
import chess
import numpy as np

# Create board.py
import chess
import numpy as np

class ChessBoard:
    def __init__(self):
        self.board = chess.Board()
        self.piece_values = {
            chess.PAWN: 1,
            chess.KNIGHT: 3,
            chess.BISHOP: 3,
            chess.ROOK: 5,
            chess.QUEEN: 9,
            chess.KING: 0
        }

    def get_board_state(self):
      """Convert board to neural network input format"""
      # Create state array with shape (13, 8, 8)
      state = np.zeros((13, 8, 8), dtype=np.float32)  # 6 piece types * 2 colors + 1 for empty

      for square in chess.SQUARES:
          piece = self.board.piece_at(square)
          if piece is None:
              state[12][square // 8][square % 8] = 1  # Empty square
          else:
              piece_idx = piece.piece_type - 1 + (6 if piece.color else 0)
              state[piece_idx][square // 8][square % 8] = 1
      print(f"Board state shape: {state.shape}")  # Should be (13, 8, 8)

      return state

    def get_legal_moves(self):
        """Get list of legal moves in UCI format"""
        return [move.uci() for move in self.board.legal_moves]

    def make_move(self, move_uci):
        """Make a move in UCI format"""
        move = chess.Move.from_uci(move_uci)
        self.board.push(move)

    def is_game_over(self):
        """Check if the game is over"""
        return self.board.is_game_over()

    def get_result(self):
        """Get game result (1 for white win, -1 for black win, 0 for draw)"""
        if not self.is_game_over():
            return None

        result = self.board.result()
        if result == "1-0":
            return 1
        elif result == "0-1":
            return -1
        else:
            return 0

In [23]:
# Create model.py
import torch
import torch.nn as nn
import torch.nn.functional as F

class ChessNet(nn.Module):
    def __init__(self):
        super(ChessNet, self).__init__()
        self.conv1 = nn.Conv2d(13, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(256 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc_value = nn.Linear(512, 1)
        self.fc_policy = nn.Linear(512, 4672)  # All possible moves

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))

        x = x.view(-1, 256 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        value = torch.tanh(self.fc_value(x))
        policy = F.log_softmax(self.fc_policy(x), dim=1)

        return value, policy

def load_model(path):
    model = ChessNet()
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

In [24]:
# Create agent.py
import torch
import torch.nn.functional as F
import numpy as np
from src.board import ChessBoard

class ChessAgent:
    def __init__(self, model):
        self.model = model
        self.board = ChessBoard()

    def select_move(self, temperature=1.0):
      """Select a move using the model"""
      state = self.board.get_board_state()  # Shape: (13, 8, 8)
      # Convert to tensor and ensure correct shape [batch_size, channels, height, width]
      state_tensor = torch.FloatTensor(state).unsqueeze(0)  # Shape: [1, 13, 8, 8]
      print(f"Input tensor shape before model: {state_tensor.shape}")

    with torch.no_grad():
        value, policy = self.model(state_tensor)  # Use state_tensor directly

        # Add batch dimension and convert to tensor
        state = torch.FloatTensor(state).unsqueeze(0)  # Shape: [1, 13, 8, 8]

        with torch.no_grad():
            value, policy = self.model(state)

        # Apply temperature to policy
        policy = policy / temperature

        # Get legal moves
        legal_moves = self.board.get_legal_moves()

        # Filter policy for legal moves
        move_probs = torch.zeros(len(legal_moves))
        for i, move in enumerate(legal_moves):
            move_idx = self._move_to_index(move)
            move_probs[i] = policy[0][move_idx]

        # Sample move
        move_probs = F.softmax(move_probs, dim=0)
        move_idx = torch.multinomial(move_probs, 1).item()

        return legal_moves[move_idx]

    def _move_to_index(self, move_uci):
        """Convert UCI move to policy index"""
        # This is a simplified version - you'll need to implement the full conversion
        from_square = chess.parse_square(move_uci[:2])
        to_square = chess.parse_square(move_uci[2:4])
        return from_square * 64 + to_square

ModuleNotFoundError: No module named 'src.board'

In [ ]:
# Create training.py
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import numpy as np
from src.board import ChessBoard
from src.model import ChessNet
from src.agent import ChessAgent

class SelfPlayTrainer:
    def __init__(self, model_path=None):
        self.model = ChessNet()
        if model_path:
            self.model.load_state_dict(torch.load(model_path))
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        self.value_criterion = nn.MSELoss()
        self.policy_criterion = nn.CrossEntropyLoss()

    def self_play_game(self, temperature=1.0):
        """Play a game against itself"""
        board = ChessBoard()
        agent = ChessAgent(self.model)
        states = []
        moves = []

        while not board.is_game_over():
            state = board.get_board_state()
            move = agent.select_move(temperature)

            states.append(state)
            moves.append(move)

            board.make_move(move)

        result = board.get_result()
        return states, moves, result

    def train_step(self, states, moves, result):
        """Train on a batch of self-play data"""
        self.model.train()
        self.optimizer.zero_grad()

        # Convert states to tensor and ensure correct shape
        states = np.array(states)  # Shape: [batch_size, 13, 8, 8]
        states = torch.FloatTensor(states)  # Keep the same shape

        moves = torch.LongTensor(moves)
        result = torch.FloatTensor([result])

        value, policy = self.model(states)
        # Rest of the code...
        # Calculate losses
        value_loss = self.value_criterion(value.squeeze(), torch.FloatTensor([result]))
        policy_loss = self.policy_criterion(policy, moves)

        # Combined loss
        loss = value_loss + policy_loss

        loss.backward()
        self.optimizer.step()

        return loss.item()

    def train(self, num_games=100, num_epochs=10):
        """Train the model through self-play"""
        for game in tqdm(range(num_games)):
            states, moves, result = self.self_play_game()

            for epoch in range(num_epochs):
                loss = self.train_step(states, moves, result)

            if (game + 1) % 10 == 0:
                torch.save(self.model.state_dict(), f'models/checkpoint_{game+1}.pt')

In [ ]:
# Create interface.py
import chess
from src.board import ChessBoard
from src.agent import ChessAgent
from src.model import load_model

class ChessCLI:
    def __init__(self, model_path):
        self.model = load_model(model_path)
        self.agent = ChessAgent(self.model)
        self.board = ChessBoard()

    def print_board(self):
        print(self.board.board)

    def play(self, human_plays_white=True):
        while not self.board.is_game_over():
            self.print_board()

            if (human_plays_white and self.board.board.turn) or \
               (not human_plays_white and not self.board.board.turn):
                # Human's turn
                move = input("Enter your move (UCI format, e.g., 'e2e4'): ")
                try:
                    self.board.make_move(move)
                except ValueError:
                    print("Invalid move! Try again.")
                    continue
            else:
                # Engine's turn
                move = self.agent.select_move()
                print(f"Engine plays: {move}")
                self.board.make_move(move)

        self.print_board()
        result = self.board.get_result()
        if result == 1:
            print("White wins!")
        elif result == -1:
            print("Black wins!")
        else:
            print("Draw!")

In [ ]:
# Training parameters
NUM_GAMES = 10
NUM_EPOCHS = 1
MODEL_SAVE_PATH = '/content/drive/MyDrive/chess_engine/models'

# Create trainer
from src.training import SelfPlayTrainer
trainer = SelfPlayTrainer()

# Train the model
print("Starting training...")
trainer.train(num_games=NUM_GAMES, num_epochs=NUM_EPOCHS)
print("Training completed!")

In [ ]:
# Download the trained model
!cp {MODEL_SAVE_PATH}/checkpoint_{NUM_GAMES}.pt /content/models/checkpoint_{NUM_GAMES}.pt
!zip -r /content/chess_model.zip /content/models/

from google.colab import files
files.download('/content/chess_model.zip')